### Extraindo dados da API do portal da transparência do programa Novo Bolsa Família

* 1º - Baixar os dados de Novo Bolsa Família dos meses 01/2024 e 03/2024 pela API;
* 2º - Filtrar os dados pelo município de Wanderlandia e verificar no mês 01/2024 quanto esse município recebeu;
    * Reposta: 972.538,00
* 3º - Contar quantos benefícios foram recebidos por UF e qual a UF que mais recebeu benefícios em 03/2024.
    * Resposta: 'SP'

In [13]:
! pip install requests
! pip install pandas

In [14]:
import requests
import pandas as pd
from zipfile import ZipFile
from io import BytesIO

### Criar função para baixar e extrair os dados

In [15]:
def baixar_e_extrair_dados(url):
    # Envia uma requisição para baixar o arquivo
    response = requests.get(url)

    # Checa se a requisição foi bem sucedida
    if response.status_code == 200:
        tamanho_bytes = len(response.content)
        tamanho_mb = tamanho_bytes / (1024 * 1024)
        print(f'Tamanho do arquivo: {tamanho_bytes}B, ({tamanho_mb:.2f}MB)')

        # Usa BytesIO para ler o arquivo ZIP da memória
        with ZipFile(BytesIO(response.content)) as arquivo_zip:
            nomes_arquivos = arquivo_zip.namelist()

            if not nomes_arquivos:
                print("O arquivo ZIP está vazio ou não contém arquivos.")
                return None

            # Pega o nome do primeiro arquivo dentro do ZIP (assumindo que seja o CSV)
            nome_arquivo_csv = nomes_arquivos[0]

            # Abre o arquivo CSV *dentro do ZIP* usando arquivo_zip.open()
            with arquivo_zip.open(nome_arquivo_csv) as arquivo_csv_interno:
                # O pandas pode ler diretamente do objeto de arquivo retornado por zipfile.open()
                # Definir o tamanho do chunk
                chunk_size = 50000 # Número de linhas a serem lidas por vez
                
                # Use pd.read_csv com o objeto de arquivo interno
                # Para evitar problemas de memória com arquivos grandes, use chunksize
                df_chunks = pd.read_csv(arquivo_csv_interno, sep=';', encoding='ISO-8859-1', chunksize=chunk_size, low_memory=False)
                
                # Concatena os chunks em um único DataFrame
                df = pd.concat(df_chunks)
                
                return df
    else:
        print(f'Falha no download do arquivo: Status: {response.status_code}')
        return None



### URL do arquivo de 01/2024

In [16]:
url = 'https://portaldatransparencia.gov.br/download-de-dados/novo-bolsa-familia/202401'

In [ ]:
# Baixar, extrair e ler os dados de 01/2024
df = baixar_e_extrair_dados(url)

KeyboardInterrupt: 

In [ ]:
df.head()
# df
# df.describe()

,MÊS COMPETÊNCIA,MÊS REFERÊNCIA,UF,CÓDIGO MUNICÍPIO SIAFI,NOME MUNICÍPIO,CPF FAVORECIDO,NIS FAVORECIDO,NOME FAVORECIDO,VALOR PARCELA
0,202401,202303,AC,139,RIO BRANCO,***.511.972-**,1.629112e+10,JANAINA FERREIRA DA SILVA,"800,00"
1,202401,202303,BA,3329,ANGUERA,***.909.745-**,2.047443e+10,TACIANE MARQUES DA SILVA,"750,00"
2,202401,202303,BA,3731,MONTE SANTO,***.872.338-**,2.019538e+10,MAILSON DOS SANTOS SILVA,"650,00"
3,202401,202303,BA,3761,NOVA VICOSA,***.665.905-**,1.272359e+10,IRENI RICARDO GERALDO,"650,00"
4,202401,202303,BA,3849,SALVADOR,***.932.435-**,2.002458e+10,ELIENE GOMES DOS SANTOS,"850,00"


In [ ]:
df_filtrado_municipio = df[df['NOME MUNICÍPIO'] == 'WANDERLANDIA']
df_filtrado_municipio

,MÊS COMPETÊNCIA,MÊS REFERÊNCIA,UF,CÓDIGO MUNICÍPIO SIAFI,NOME MUNICÍPIO,CPF FAVORECIDO,NIS FAVORECIDO,NOME FAVORECIDO,VALOR PARCELA
72094,202401,202310,TO,9665,WANDERLANDIA,***.729.691-**,1.618199e+10,GILDAMAR SINHA BENIGNO,"810,00"
121566,202401,202311,TO,9665,WANDERLANDIA,***.729.691-**,1.618199e+10,GILDAMAR SINHA BENIGNO,"810,00"
175754,202401,202312,TO,9665,WANDERLANDIA,***.729.691-**,1.618199e+10,GILDAMAR SINHA BENIGNO,"810,00"
20830263,202401,202401,TO,9665,WANDERLANDIA,***.688.783-**,2.091861e+10,ABDIAS SOARES SANTANA,"600,00"
20830264,202401,202401,TO,9665,WANDERLANDIA,***.826.721-**,2.122337e+10,ACEONE JOSE DE FREITAS,"455,00"
...,...,...,...,...,...,...,...,...,...
20831724,202401,202401,TO,9665,WANDERLANDIA,NaN,1.633356e+10,YASMIM SOARES SILVA,"475,00"
20831725,202401,202401,TO,9665,WANDERLANDIA,***.540.251-**,2.061975e+10,ZAIRA FERREIRA DA SILVA PEREIRA,"490,00"
20831726,202401,202401,TO,9665,WANDERLANDIA,***.606.461-**,1.653206e+10,ZELIA DAMACENO MATOS,"700,00"
20831727,202401,202401,TO,9665,WANDERLANDIA,***.493.771-**,1.653523e+10,ZILDETE DOS SANTOS PAZ,"600,00"


### Tentativa errada (o campo está vindo com vírgula e não ponto)

In [23]:
soma_parcelas = df_filtrado_municipio['VALOR PARCELA'].sum()
soma_parcelas

'810,00810,00810,00600,00455,00600,00750,00600,00300,00650,00816,00600,00650,00650,00600,00205,00716,00992,00800,00750,00540,00325,00600,00900,001060,001002,00650,00766,00750,00600,00800,00600,00650,00650,00600,00440,00205,00910,00600,00600,00750,00600,00600,00750,00288,00750,00600,00750,00850,00750,00616,00640,00750,00900,00600,00650,00600,00600,00800,00750,00590,00750,00650,00650,00600,00850,00750,00750,00750,00750,00750,00840,00650,00790,00400,00576,00750,00600,00900,00650,00800,00440,00850,00900,00600,00650,00950,00490,001110,00600,00490,00590,00640,00600,00440,00950,001060,00800,00650,00325,00700,00600,00640,00640,00900,00650,00650,00750,00600,00470,00750,00600,00650,00800,00750,001000,00490,00600,00750,00750,00650,00440,00600,00400,00700,00490,00750,00750,00950,00600,00600,00450,00750,00750,00600,00325,00750,00511,00750,00750,00750,00750,00375,00600,00950,00700,00600,00600,00650,001202,00750,00600,00440,00600,00650,00600,00650,00600,00600,00650,00600,00300,00750,00600,00700,00375

### Tentativa correta

In [24]:
soma_parcelas = df_filtrado_municipio['VALOR PARCELA'].str.replace(',', '.').astype(float)
total = soma_parcelas.sum()
total

972538.0

### URL do arquivo de março

In [26]:
url = 'https://portaldatransparencia.gov.br/download-de-dados/novo-bolsa-familia/202403'

In [27]:
# Baixar, extrair e ler os dados de 03/2024
df = baixar_e_extrair_dados(url)

Tamanho do arquivo: 365124681B, (348.21MB)


In [30]:
df_filtrado_uf = df['UF']
df_filtrado_uf

0           PA
1           PA
2           AM
3           BA
4           BA
            ..
20534085    TO
20534086    TO
20534087    TO
20534088    TO
20534089    TO
Name: UF, Length: 20534090, dtype: object

In [32]:
df_frequencia_uf = df_filtrado_uf.value_counts()
df_frequencia_uf

UF
SP    2493243
BA    2445216
RJ    1673124
PE    1576690
MG    1575003
CE    1455369
PA    1328423
MA    1203858
PB     666232
AM     634141
RS     609188
PR     597560
PI     596840
AL     535105
GO     504482
RN     499842
SE     378152
ES     299399
MT     254439
SC     223305
MS     204809
DF     185046
TO     156352
AC     128859
RO     121860
AP     115126
RR      72427
Name: count, dtype: int64

In [33]:
df_frequencia_formatada = df_frequencia_uf.to_frame().style.format('{:,}')
df_frequencia_formatada

,count
UF,
SP,"2,493,243"
BA,"2,445,216"
RJ,"1,673,124"
PE,"1,576,690"
MG,"1,575,003"
CE,"1,455,369"
PA,"1,328,423"
MA,"1,203,858"
PB,"666,232"


In [34]:
df_uf_mais_frequente = df_frequencia_uf.idxmax()
df_uf_mais_frequente

'SP'